In [1]:
from collections import defaultdict
import numpy as np
import pandas as pd
import scipy.io as io
import gzip
import math
import os
import re
import scipy
from utils import *

In [ ]:
#NOTE: don't execute this cell
#TODO: still need to understand why those lines in Maria's code are necessary

zurich['BNCfreq'] = zurich.WORDstrip.map(lambda x: unigrdict.get(str(x).lower()))
zurich.BNCfreq = zurich.BNCfreq.fillna(zurich.BNCfreq.min())
zurich.BNCfreq = zurich.BNCfreq/100 #because 100 million word - to get freq per million
zurich.BNCfreq = np.log(zurich.BNCfreq)

zurich['BNCfreqinv']= -zurich.BNCfreq

In [2]:
# instantiate data transformer object for task 1 (NR) on sentence level with min-max feature scaling
datatransform = DataTransformer('task1', level='sentence', scaling='min-max')

In [3]:
# get data for all subjects according to settings in cell above
# NOTE: data for each sbj will be stored in a pd.DataFrame
sbjs = [datatransform(i) for i in range(12)]

C:\Users\Lukas\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\frame.py:4034: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)
C:\Users\Lukas\Documents\GitHub\NER-with-EEG-and-ET\utils.py:172: RuntimeWarning: invalid value encountered in true_divide
  features = np.array([(feat - min(feat))/(max(feat) - min(feat)) for feat in features.T])


In [ ]:
# check whether data is complete
for i, sbj in enumerate(sbjs):
    print("Subject:", i)
    print()
    print(len(sbj))
    print()